#Configurações e Instalação de pacotes

In [0]:
%pip install --quiet --upgrade scikit-learn==1.4.2 tensorflow==2.16.1 scikeras==0.13.0 joblib

print("Dependências de MLOps instaladas/verificadas.")
print("Versões compatíveis: scikit-learn==1.4.2, tensorflow==2.16.1, scikeras==0.13.0")
print("Reiniciando o kernel Python agora para carregar as novas bibliotecas...")


# Esta função nativa do Databricks reinicia o kernel Python.
# Todas as variáveis serão limpas, o que é o comportamento esperado 
# ao instalar bibliotecas.

In [0]:
%restart_python

# Configuração

In [0]:
# --- Imports ---
import numpy as np
import pandas as pd
import tensorflow as tf
import mlflow
import joblib
import json
import os
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, recall_score, precision_score, f1_score
from sklearn.impute import SimpleImputer
from tensorflow.keras.callbacks import EarlyStopping
from mlflow.models.signature import infer_signature
from datetime import datetime

# --- Configurações Globais ---
# A View de features que já existe e será lida
VIEW_NAME = "transacoes_db.feature_store.in_live_features"

# --- MUDANÇA PRINCIPAL AQUI ---
# Diretório em seu Volume do Unity Catalog para salvar os artefatos
# (Assumindo que você queira manter uma subpasta 'pix_fraud' dentro do volume)
MODEL_ARTIFACTS_DIR = "/Volumes/transacoes_db/copper/files/pix_fraud/production_artifacts/"
# --- FIM DA MUDANÇA ---

# Semente para reprodutibilidade
SEED = 42

# --- Configurações de Ambiente ---
warnings.filterwarnings('ignore')

# Garantir que o diretório de artefatos exista no Volume
os.makedirs(MODEL_ARTIFACTS_DIR, exist_ok=True)
print(f"Diretório de artefatos garantido: {MODEL_ARTIFACTS_DIR}")

# --- Sementes de Aleatoriedade ---
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Inicializar SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
%sql
/* CONTEXTO DA VIEW: transacoes_db.feature_store.in_live_features
(Esta célula %sql é apenas para documentação, a view já existe)

Esta view agrega dados transacionais com perfis e features de janela temporal.
*
SELECT
  -- Colunas da transação
  ft.valor AS valor_transacao,
  ft.data AS data_transacao,
  ft.id_conta_origem AS id_conta_pagador,
  ft.id_conta_destino AS id_conta_recebedor,
  ft.id_tipo_iniciacao_pix AS tipo_iniciacao_pix_id,
  ft.id_finalidade_pix AS finalidade_pix_id,
  
  -- Targets (Rótulos)
  ft.is_fraud AS transacao_fraudulenta, -- (Binário: 0 ou 1)
  ft.fraud_type AS tipo_fraude,         -- (Multiclasse: 'scam', 'roubo', etc.)

  -- Perfis (Pagador e Recebedor)
  conta_orig.saldo AS pagador_saldo,
  conta_orig.aberta_em AS pagador_conta_aberta_em,
  conta_orig.id_tipo_conta AS pagador_tipo_conta_id,
  cliente_orig.id_natureza AS pagador_natureza_id,
  cliente_orig.nascido_em AS pagador_data_nascimento,
  conta_dest.saldo AS recebedor_saldo,
  conta_dest.aberta_em AS recebedor_conta_aberta_em,
  conta_dest.id_tipo_conta AS recebedor_tipo_conta_id,
  cliente_dest.id_natureza AS recebedor_natureza_id,
  cliente_dest.nascido_em AS recebedor_data_nascimento,

  -- Features de Tempo Real (Window Functions)
  ft.pagador_txs_ultimas_24h,
  ft.pagador_valor_ultimas_24h,
  ft.recebedor_txs_ultima_1h,
  ft.recebedor_valor_ultima_1h,
  ft.pagador_segundos_desde_ultima_tx
FROM
  transacoes_db.feature_store.in_live_features AS ft
... (JOINS com tabelas copper de perfis) ...
limit 1 */


## Carregamento e Pre-Processamento de dados

In [0]:
print(f"Carregando dados da view: {VIEW_NAME}...")
# 1. Carregar dados do Delta Lake (Spark)
df_spark = spark.read.table(VIEW_NAME)

# 2. Converter para Pandas
# Assumindo que o dataset cabe na memória do driver para treinamento com Sklearn/Keras
df_pandas = df_spark.toPandas()
print(f"Dados carregados. Total de {len(df_pandas)} registros.")

# 3. Separar Features (X) e Targets (y)
# IDs não são features para o modelo
features_to_drop = ['transacao_fraudulenta', 'tipo_fraude', 'id_conta_pagador', 'id_conta_recebedor']
X = df_pandas.drop(columns=features_to_drop)
y_binary = df_pandas['transacao_fraudulenta']
y_multiclass = df_pandas['tipo_fraude'] # Este ainda é string ('scam', 'legitimo', etc.)

# 4. Divisão em Treino (70%), Validação (15%) e Teste (15%)
# Primeiro, 70% treino e 30% temporário (para val/teste)
X_train, X_temp, y_train_binary, y_temp_binary, y_train_multiclass, y_temp_multiclass = train_test_split(
    X, y_binary, y_multiclass, 
    test_size=0.30, 
    random_state=SEED, 
    stratify=y_binary # Garantir proporção de fraude
)

# Segundo, dividir os 30% temporários em 15% validação e 15% teste (50% de 30% = 15%)
X_val, X_test, y_val_binary, y_test_binary, y_val_multiclass, y_test_multiclass = train_test_split(
    X_temp, y_temp_binary, y_temp_multiclass, 
    test_size=0.50, # 50% do temp (que era 30% do total)
    random_state=SEED, 
    stratify=y_temp_binary # Garantir proporção de fraude
)

# Verificar as proporções
print("--- Divisão dos Dados ---")
print(f"Treino:   {X_train.shape[0]} amostras")
print(f"Validação: {X_val.shape[0]} amostras")
print(f"Teste:    {X_test.shape[0]} amostras")

print("\n--- Proporção de Fraude (Binário) ---")
print(f"Treino:    {y_train_binary.value_counts(normalize=True)[1]:.4f}")
print(f"Validação: {y_val_binary.value_counts(normalize=True)[1]:.4f}")
print(f"Teste:     {y_test_binary.value_counts(normalize=True)[1]:.4f}")

#Funções para Treinamento do Modelo / Definição de Features


In [0]:
# --- Funções Customizadas para Engenharia de Features de Data/Hora ---

def feature_engineer_datetimes(df_in: pd.DataFrame) -> pd.DataFrame:
    """
    Recebe um DataFrame com as colunas de data/hora e retorna
    um DataFrame com features numéricas de engenharia.
    """
    # Criar cópia para evitar SettingWithCopyWarning
    df = df_in.copy()
    
    # DataFrame de saída
    df_out = pd.DataFrame(index=df.index)
    
    # Timestamp atual para cálculos de idade
    now = datetime.now()

    # 1. 'data_transacao'
    dt_tx = pd.to_datetime(df['data_transacao'])
    df_out['tx_hora_do_dia'] = dt_tx.dt.hour
    df_out['tx_dia_da_semana'] = dt_tx.dt.dayofweek
    df_out['tx_mes'] = dt_tx.dt.month

    # 2. Idade das Contas (em dias)
    # df_out['pagador_idade_conta_dias'] = (now - pd.to_datetime(df['pagador_conta_aberta_em'])).dt.days    # REMOVIDO: Agora vem diretamente do SQL
    df_out['recebedor_idade_conta_dias'] = (now - pd.to_datetime(df['recebedor_conta_aberta_em'])).dt.days
    
    # 3. Idade dos Clientes (em anos)
    df_out['pagador_idade_anos'] = ((now - pd.to_datetime(df['pagador_data_nascimento'])).dt.days / 365.25).astype(int)
    df_out['recebedor_idade_anos'] = ((now - pd.to_datetime(df['recebedor_data_nascimento'])).dt.days / 365.25).astype(int)

    # Lidar com possíveis nulos que podem ter sido gerados (ex: datas inválidas)
    df_out = df_out.fillna(0)
    
    return df_out

# --- Definição das Listas de Colunas ---

# Colunas numéricas que entram direto no scaler
numeric_features = [
    'valor_transacao', 
    'pagador_saldo', 
    'recebedor_saldo',
    'pagador_txs_ultimas_24h',
    'pagador_valor_ultimas_24h',
    'recebedor_txs_ultima_1h',
    'recebedor_valor_ultima_1h',
    'pagador_segundos_desde_ultima_tx',
    
    # --- Novas Features (Adicionadas do SQL) ---
    'primeira_interacao',
    'pagador_interacoes_com_recebedor',
    'recebedor_num_pagadores_unicos_24h',
    'recebedor_idade_conta_dias',
    'pagador_idade_conta_dias',
    'valor_vs_media_pagador_30d',
    'valor_vs_saldo_pagador'
]

# Colunas categóricas para One-Hot Encoding
categorical_features = [
    'tipo_iniciacao_pix_id', 
    'finalidade_pix_id', 
    'pagador_tipo_conta_id', 
    'pagador_natureza_id',
    'recebedor_tipo_conta_id', 
    'recebedor_natureza_id'
]

# Colunas de data/hora para o transformador customizado
datetime_features = [
    'data_transacao', 
    'pagador_conta_aberta_em', 
    'pagador_data_nascimento',
    'recebedor_conta_aberta_em', 
    'recebedor_data_nascimento'
]

# --- Criação dos Pipelines de Transformação ---

# Pipeline para features numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # ADICIONADO para tratar NULLs
    ('scaler', StandardScaler())
])

# Pipeline para features categóricas
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pipeline para features de data/hora
datetime_transformer = Pipeline(steps=[
    ('feature_eng', FunctionTransformer(feature_engineer_datetimes)),
    ('scaler', StandardScaler()) # Escalonar as features de data/hora criadas
])

# --- Montagem do ColumnTransformer (Pré-processador) ---

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('date', datetime_transformer, datetime_features)
    ],
    remainder='drop' # Ignorar colunas não listadas (ex: IDs)
)

print("Pipeline de pré-processamento 'preprocessor' definido com sucesso.")

## Funções de Criação de modelo Keras

In [0]:
# --- Função Factory para Modelo Binário ---
def build_binary_model(meta):
    """Constrói o modelo binário para o KerasClassifier."""
    # scikeras injeta n_features_in_ automaticamente
    n_features_in = meta["n_features_in_"]
    
    # Garantir que a semente do TF seja definida dentro da função
    # para reprodutibilidade quando o KerasClassifier for clonado
    tf.random.set_seed(SEED) 

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(n_features_in,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid') # Saída sigmoide para binário
    ])

    model.compile(
        loss='binary_crossentropy', 
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
        metrics=['AUC', tf.keras.metrics.Recall(name='recall'), tf.keras.metrics.Precision(name='precision')]
    )
    return model

# --- Função Factory para Modelo Multiclasse ---
def build_multiclass_model(meta):
    """Constrói o modelo multiclasse para o KerasClassifier."""
    n_features_in = meta["n_features_in_"]
    # scikeras injeta o número de classes únicas (ex: 3 tipos de fraude)
    n_classes_out = meta["n_classes_"]
    
    tf.random.set_seed(SEED)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(n_features_in,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(n_classes_out, activation='softmax') # Saída softmax
    ])

    model.compile(
        loss='sparse_categorical_crossentropy', # Usar sparse pois os labels são inteiros (0, 1, 2)
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
        metrics=['accuracy']
    )
    return model

print("Funções de construção de modelo Keras ('build_binary_model', 'build_multiclass_model') definidas.")

# Treinamento do modelo Binario 

In [0]:
# Definir callback de Early Stopping
# Parar o treino se a perda na validação (val_loss) não melhorar após 5 épocas
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

# 1. Iniciar experimento MLflow
mlflow.set_experiment(f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/Pix_Fraud_Detection")

with mlflow.start_run(run_name="Fraud_Binary_Classifier") as run_binary:
    print(f"Iniciando Run MLflow: {run_binary.info.run_id}")
    
    # --- CORREÇÃO AQUI ---
    # O 'scikeras.wrappers' moderno usa o argumento 'model' e não 'build_fn'
    keras_binary_model = KerasClassifier(
        model=build_binary_model,  # <-- MUDANÇA DE 'build_fn' PARA 'model'
        epochs=50, 
        batch_size=256, 
        verbose=1,
        random_state=SEED
    )
    # --- FIM DA CORREÇÃO ---

    # 3. Criar o Pipeline completo
    pipeline_binary = Pipeline([
        ('preprocessor', preprocessor), 
        ('model', keras_binary_model)
    ])

    # 4. Pré-processar dados de validação para Early Stopping
    # O pipeline.fit não transforma automaticamente os dados passados para 'model__validation_data'
    print("Pré-ajustando o processador para transformar dados de validação...")
    # Usamos clone() para não "sujar" o preprocessor do pipeline antes do .fit()
    preprocessor_for_val = clone(preprocessor).fit(X_train)
    X_val_processed = preprocessor_for_val.transform(X_val)
    print(f"Dimensões dos dados de validação processados: {X_val_processed.shape}")
    
    # 5. Treinar o pipeline
    print("Iniciando treinamento do pipeline binário...")
    pipeline_binary.fit(
        X_train, y_train_binary, 
        model__validation_data=(X_val_processed, y_val_binary), 
        model__callbacks=[early_stopping]
    )
    print("Treinamento binário concluído.")

    # 6. Avaliar no set de teste
    y_pred_binary_proba = pipeline_binary.predict_proba(X_test)[:, 1]
    y_pred_binary = (y_pred_binary_proba >= 0.5).astype(int)

    # 7. Logar métricas (Foco em Recall)
    recall = recall_score(y_test_binary, y_pred_binary)
    precision = precision_score(y_test_binary, y_pred_binary)
    f1 = f1_score(y_test_binary, y_pred_binary)
    roc_auc = roc_auc_score(y_test_binary, y_pred_binary_proba)

    mlflow.log_metric("test_recall", recall)
    mlflow.log_metric("test_precision", precision)
    mlflow.log_metric("test_f1", f1)
    mlflow.log_metric("test_roc_auc", roc_auc)
    
    print("\n--- Métricas de Teste (Binário) ---")
    print(f"Recall:    {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"ROC AUC:   {roc_auc:.4f}")
    print(classification_report(y_test_binary, y_pred_binary))

    # 8. Salvamento Crítico do Artefato (usará o caminho do Volume)
    binary_pipeline_path = os.path.join(MODEL_ARTIFACTS_DIR, "fraud_binary_pipeline.pkl")
    joblib.dump(pipeline_binary, binary_pipeline_path)
    print(f"Pipeline binário salvo em: {binary_pipeline_path}")

    # 9. Logar no MLflow (modelo e artefato)
    signature = infer_signature(X_train, pipeline_binary.predict(X_train))
    mlflow.sklearn.log_model(
        pipeline_binary, 
        "binary_model", 
        signature=signature,
        input_example=X_train.head(5).to_dict(orient='records')
    )
    mlflow.log_artifact(binary_pipeline_path)

print("Célula 6 concluída.")

## Analise Visual

In [0]:
from sklearn.model_selection import learning_curve
import numpy as np
import matplotlib.pyplot as plt

# 1. Definição da Função (Já ajustada para imprimir resultados)
def plot_learning_curve(model, X, y, title="Curva de Aprendizado", 
                        cv=5, 
                        train_sizes=np.linspace(0.1, 1.0, 5),
                        scoring='f1'): # Adicionado 'scoring' como parâmetro para clareza
    """
    Plota a curva de aprendizado e imprime os scores médios.
    """
    
    # Informa quantos treinamentos serão feitos
    total_fits = len(train_sizes) * cv
    print(f"Calculando curva de aprendizado... Total de {total_fits} treinamentos.")

    # A função learning_curve retorna os tamanhos absolutos usados
    train_sizes_abs, train_scores, val_scores = learning_curve(
        model, X, y, cv=cv, scoring=scoring, n_jobs=-1,
        train_sizes=train_sizes, shuffle=True, random_state=42
    )

    train_mean = np.mean(train_scores, axis=1)
    val_mean = np.mean(val_scores, axis=1)

    # --- O "Resultado Direto" ---
    print("\n--- Resultados (Médias) ---")
    print(f"{'Tamanho Treino':<15} | {'F1 Treino':<10} | {'F1 Validação':<10}")
    print("-" * 47)
    for i, size in enumerate(train_sizes_abs):
        print(f"{size:<15} | {train_mean[i]:<10.4f} | {val_mean[i]:<10.4f}")
    print("---------------------------\n")
    # -----------------------------

    plt.figure(figsize=(8,6))
    plt.plot(train_sizes_abs, train_mean, 'o-', label="Treino", linewidth=2)
    plt.plot(train_sizes_abs, val_mean, 'o-', label="Validação", linewidth=2)
    plt.title(title, fontsize=14)
    plt.xlabel("Tamanho do conjunto de treino (Nº de amostras)")
    plt.ylabel("F1-Score")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

# --- 2. Definição dos parâmetros para rodar rápido ---
cv_rapido = 3
tamanhos_rapidos = np.linspace(0.25, 1.0, 3) # 3 pontos (25%, 62.5%, 100%)




In [0]:
# plot_learning_curve(
#     pipeline_binary, 
#     X_train, 
#     y_train_binary, 
#     title="Curva de Aprendizado - Modelo Binário (Rápida)", 
#     cv=cv_rapido, 
#     train_sizes=tamanhos_rapidos,
#     scoring='f1' # Scoring correto para o binário
# )

In [0]:
# 📊 COMPARATIVO DE MÉTRICAS ENTRE TREINO E TESTE - MODELO BINÁRIO
import seaborn as sns
import pandas as pd

# Calcular métricas no treino
y_pred_train = pipeline_binary.predict(X_train)
y_pred_train_proba = pipeline_binary.predict_proba(X_train)[:, 1]

recall_train = recall_score(y_train_binary, y_pred_train)
precision_train = precision_score(y_train_binary, y_pred_train)
f1_train = f1_score(y_train_binary, y_pred_train)
roc_auc_train = roc_auc_score(y_train_binary, y_pred_train_proba)

metrics_data = {
    'Conjunto': ['Treino', 'Teste'],
    'Recall': [recall_train, recall],
    'Precision': [precision_train, precision],
    'F1-Score': [f1_train, f1],
    'ROC AUC': [roc_auc_train, roc_auc]
}

df_metrics = pd.DataFrame(metrics_data)
df_metrics_melt = df_metrics.melt(id_vars='Conjunto', var_name='Métrica', value_name='Valor')

plt.figure(figsize=(8,6))
sns.barplot(x='Métrica', y='Valor', hue='Conjunto', data=df_metrics_melt, palette='viridis')
plt.title('Comparativo de Métricas - Treino vs Teste (Modelo Binário)', fontsize=14)
plt.ylim(0.85, 1.0)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


In [0]:
# 📈 DISPERSÃO DAS PROBABILIDADES PREDITAS - MODELO BINÁRIO
plt.figure(figsize=(8,6))
sns.scatterplot(x=range(len(y_test_binary)), y=y_pred_binary_proba,
                hue=y_test_binary, palette=['#2ecc71','#e74c3c'], alpha=0.6)
plt.title('Dispersão das Probabilidades Previstas - Modelo Binário', fontsize=14)
plt.xlabel('Índice da Amostra')
plt.ylabel('Probabilidade Prevista de Fraude')
plt.legend(title='Classe Real', labels=['Não Fraude','Fraude'])
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


In [0]:
# 🧩 MATRIZ DE CONFUSÃO ESTILIZADA - MODELO BINÁRIO
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test_binary, y_pred_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(6,6))
disp.plot(cmap='Blues', values_format='d', ax=ax, colorbar=False)
plt.title('Matriz de Confusão - Modelo Binário')
plt.show()


# Treinamento do Modelo Multiclasse

In [0]:
print("Iniciando preparação para o Modelo 2 (Multiclasse)...")

# 1. Filtrar dados para incluir apenas transações fraudulentas
train_fraud_mask = (y_train_binary == 1)
val_fraud_mask = (y_val_binary == 1)
test_fraud_mask = (y_test_binary == 1)

X_train_fraud = X_train[train_fraud_mask]
y_train_multiclass_fraud = y_train_multiclass[train_fraud_mask]

X_val_fraud = X_val[val_fraud_mask]
y_val_multiclass_fraud = y_val_multiclass[val_fraud_mask]

X_test_fraud = X_test[test_fraud_mask]
y_test_multiclass_fraud = y_test_multiclass[test_fraud_mask]

print(f"Total de amostras de fraude para treino: {len(X_train_fraud)}")
print(f"Total de amostras de fraude para validação: {len(X_val_fraud)}")
print(f"Total de amostras de fraude para teste: {len(X_test_fraud)}")

# 2. Usar LabelEncoder para converter labels string em inteiros
label_encoder = LabelEncoder()
y_train_multiclass_encoded = label_encoder.fit_transform(y_train_multiclass_fraud)
y_val_multiclass_encoded = label_encoder.transform(y_val_multiclass_fraud)
y_test_multiclass_encoded = label_encoder.transform(y_test_multiclass_fraud)

print(f"Classes de fraude encontradas: {label_encoder.classes_}")

# 3. Salvamento Crítico (Mapeamento de Labels) (usará o caminho do Volume)
# Criar mapa {0: 'scam', 1: 'roubo', 2: 'lavagem'}
label_map = {i: str(cls) for i, cls in enumerate(label_encoder.classes_)}
label_map_path = os.path.join(MODEL_ARTIFACTS_DIR, "fraud_type_label_map.json")

with open(label_map_path, 'w') as f:
    json.dump(label_map, f, indent=4)
print(f"Mapa de labels salvo em: {label_map_path}")

# --- Treinamento do Modelo Multiclasse ---

# Callback para o modelo multiclasse
early_stopping_multi = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

with mlflow.start_run(run_name="Fraud_Type_Classifier") as run_multiclass:
    print(f"Iniciando Run MLflow: {run_multiclass.info.run_id}")
    

    keras_multiclass_model = KerasClassifier(
        model=build_multiclass_model, # <-- MUDANÇA DE 'build_fn' PARA 'model'
        epochs=50, 
        batch_size=128, 
        verbose=1,
        random_state=SEED
    )
    # --- FIM DA CORREÇÃO ---

    # 5. Criar o Pipeline completo
    pipeline_multiclass = Pipeline([
        ('preprocessor', preprocessor), 
        ('model', keras_multiclass_model)
    ])
    
    # 6. Pré-processar dados de validação (apenas de fraude)
    print("Pré-ajustando o processador (multiclasse) nos dados de treino de fraude...")
    # Usamos clone() para não "sujar" o preprocessor do pipeline antes do .fit()
    preprocessor_multi_for_val = clone(preprocessor).fit(X_train_fraud)
    X_val_fraud_processed = preprocessor_multi_for_val.transform(X_val_fraud)
    print(f"Dimensões dos dados de validação (fraude) processados: {X_val_fraud_processed.shape}")

    # 7. Treinar o pipeline (APENAS com dados de fraude)
    print("Iniciando treinamento do pipeline multiclasse...")
    pipeline_multiclass.fit(
        X_train_fraud, y_train_multiclass_encoded, 
        model__validation_data=(X_val_fraud_processed, y_val_multiclass_encoded), 
        model__callbacks=[early_stopping_multi]
    )
    print("Treinamento multiclasse concluído.")

    # 8. Avaliar no set de teste (filtrado)
    y_pred_multiclass_encoded = pipeline_multiclass.predict(X_test_fraud)
    
    print("\n--- Métricas de Teste (Multiclasse) ---")
    report_text = classification_report(
        y_test_multiclass_encoded, 
        y_pred_multiclass_encoded, 
        target_names=label_encoder.classes_
    )
    print(report_text)
    
    # Logar métricas
    mlflow.log_text(report_text, "classification_report.txt")
    f1_micro = f1_score(y_test_multiclass_encoded, y_pred_multiclass_encoded, average='micro')
    mlflow.log_metric("test_f1_micro", f1_micro)

    # 9. Salvamento Crítico (usará o caminho do Volume)
    multiclass_pipeline_path = os.path.join(MODEL_ARTIFACTS_DIR, "fraud_type_pipeline.pkl")
    joblib.dump(pipeline_multiclass, multiclass_pipeline_path)
    print(f"Pipeline multiclasse salvo em: {multiclass_pipeline_path}")

    # 10. Logar no MLflow
    signature_multi = infer_signature(X_train_fraud, pipeline_multiclass.predict(X_train_fraud))
    mlflow.sklearn.log_model(
        pipeline_multiclass, 
        "multiclass_model", 
        signature=signature_multi,
        input_example=X_train_fraud.head(5).to_dict(orient='records')
    )
    mlflow.log_artifact(multiclass_pipeline_path)
    mlflow.log_artifact(label_map_path) # Importante logar o mapa também

print("Célula 7 concluída.")

## Analise Visual

In [0]:
# plot_learning_curve(
#     pipeline_multiclass, 
#     X_train, 
#     y_train_binary, 
#     title="urva de Aprendizado - Modelo Multiclasse", 
#     cv=cv_rapido, 
#     train_sizes=tamanhos_rapidos
#     #scoring='f1' # Scoring correto para o binário
# )

In [0]:
# 📊 COMPARATIVO DE MÉTRICAS ENTRE TREINO E TESTE - MODELO MULTICLASSE
import seaborn as sns
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score

# Previsões no treino
y_pred_train_multiclass = pipeline_multiclass.predict(X_train_fraud)

# Métricas no treino
recall_train = recall_score(y_train_multiclass_encoded, y_pred_train_multiclass, average='macro')
precision_train = precision_score(y_train_multiclass_encoded, y_pred_train_multiclass, average='macro')
f1_train = f1_score(y_train_multiclass_encoded, y_pred_train_multiclass, average='macro')

# Métricas no teste (usando variáveis já calculadas)
recall_test = recall_score(y_test_multiclass_encoded, y_pred_multiclass_encoded, average='macro')
precision_test = precision_score(y_test_multiclass_encoded, y_pred_multiclass_encoded, average='macro')
f1_test = f1_score(y_test_multiclass_encoded, y_pred_multiclass_encoded, average='macro')

metrics_data = {
    'Conjunto': ['Treino', 'Teste'],
    'Recall (Macro)': [recall_train, recall_test],
    'Precision (Macro)': [precision_train, precision_test],
    'F1-Score (Macro)': [f1_train, f1_test]
}

df_metrics = pd.DataFrame(metrics_data)
df_metrics_melt = df_metrics.melt(id_vars='Conjunto', var_name='Métrica', value_name='Valor')

plt.figure(figsize=(8,6))
sns.barplot(x='Métrica', y='Valor', hue='Conjunto', data=df_metrics_melt, palette='mako')
plt.title('Comparativo de Métricas - Treino vs Teste (Modelo Multiclasse)', fontsize=14)
plt.ylim(0.7, 1.0)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


In [0]:
# 📈 DISPERSÃO DAS PREDIÇÕES POR CLASSE - MODELO MULTICLASSE
from sklearn.decomposition import PCA

# Reduz dimensionalidade para 2D para visualização
X_test_fraud_transformed = pipeline_multiclass.named_steps['preprocessor'].transform(X_test_fraud)
pca = PCA(n_components=2, random_state=42)
X_2d = pca.fit_transform(X_test_fraud_transformed)

plt.figure(figsize=(8,6))
sns.scatterplot(x=X_2d[:,0], y=X_2d[:,1], hue=[label_encoder.classes_[i] for i in y_pred_multiclass_encoded],
                palette='tab10', alpha=0.7)
plt.title('Dispersão das Predições por Tipo de Fraude (PCA 2D)', fontsize=14)
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Classe Prevista', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


In [0]:
# 🧩 MATRIZ DE CONFUSÃO - MODELO MULTICLASSE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm_multi = confusion_matrix(y_test_multiclass_encoded, y_pred_multiclass_encoded)
fig, ax = plt.subplots(figsize=(8,7))
disp = ConfusionMatrixDisplay(confusion_matrix=cm_multi, display_labels=label_encoder.classes_)
disp.plot(cmap='Purples', ax=ax, colorbar=False)
plt.title('Matriz de Confusão - Modelo Multiclasse')
plt.xticks(rotation=30)
plt.show()


# Avaliação Final

In [0]:
print("--- Iniciando Avaliação Final (Pós-Treino) com Artefatos Salvos ---")

# 1. Carregar artefatos do Volume
try:
    loaded_pipeline_binary = joblib.load(os.path.join(MODEL_ARTIFACTS_DIR, "fraud_binary_pipeline.pkl"))
    loaded_pipeline_multiclass = joblib.load(os.path.join(MODEL_ARTIFACTS_DIR, "fraud_type_pipeline.pkl"))
    
    map_path = os.path.join(MODEL_ARTIFACTS_DIR, "fraud_type_label_map.json")
    with open(map_path, 'r') as f:
        # Converter chaves JSON (string) de volta para inteiros
        loaded_label_map_str = json.load(f)
        loaded_label_map = {int(k): v for k, v in loaded_label_map_str.items()}
        
    print("Artefatos (2 pipelines, 1 mapa) carregados com sucesso do Volume.")

except Exception as e:
    print(f"Erro fatal ao carregar artefatos do Volume: {e}")
    # Se falhar aqui, não podemos continuar
    raise e

# --- Avaliação Modelo 1: Classificação Binária ---
print("\n--- Avaliação Modelo 1: Classificação Binária (em X_test) ---")

# Usar os dados de teste reais (y_test_binary)
y_pred_binary_loaded = loaded_pipeline_binary.predict(X_test)

print(classification_report(y_test_binary, y_pred_binary_loaded, target_names=['Legitimo', 'Fraude']))
print("Matriz de Confusão (Binário):")
print(confusion_matrix(y_test_binary, y_pred_binary_loaded))


# --- Avaliação Modelo 2: Classificação Multiclasse ---
print("\n--- Avaliação Modelo 2: Classificação Multiclasse (em X_test ONDE y_test==1) ---")

# Filtrar o X_test para incluir apenas as fraudes VERDADEIRAS
# (para avaliar o quão bem o 2º modelo classifica os tipos de fraude reais)
X_test_true_fraud = X_test[y_test_binary == 1]

# --- CORREÇÃO AQUI ---
# Usamos 'y_test_multiclass' (o split de teste)
# Em vez de 'y_multiclass' (o dataset completo)
y_test_true_fraud_labels = y_test_multiclass[y_test_binary == 1]
# --- FIM DA CORREÇÃO ---


if not X_test_true_fraud.empty:
    # Prever os índices (0, 1, 2)
    y_pred_multiclass_idx = loaded_pipeline_multiclass.predict(X_test_true_fraud)
    
    # Mapear os índices de volta para os labels string ('scam', 'roubo')
    y_pred_multiclass_labels = [loaded_label_map.get(idx, 'unknown') for idx in y_pred_multiclass_idx]
    
    # Classes reais para o relatório
    true_labels_list = list(loaded_label_map.values())
    
    print(classification_report(y_test_true_fraud_labels, y_pred_multiclass_labels, labels=true_labels_list))
    print("Matriz de Confusão (Multiclasse):")
    print(confusion_matrix(y_test_true_fraud_labels, y_pred_multiclass_labels, labels=true_labels_list))
else:
    print("Não foram encontradas fraudes verdadeiras no conjunto de teste para avaliar o modelo multiclasse.")

print("\nCélula 8 concluída. Avaliação final completa.")